In [ ]:
### Script by Johannes Cunow
### Edited: 25.01.24

# adapted from: https://github.com/OpenStitching/stitching_tutorial/blob/master/docs/Stitching%20Tutorial.md
import os
import cv2

def list_files_with_pattern(directory_path, file_pattern):
    file_paths = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file_pattern in file:
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
    return file_paths

In [ ]:
###### change if needed ######
##############################

#year= str(list_unique_names_within_range(imgs,52,56))
year = '2025'
#maxdepth = list_unique_names_within_range(imgs,62,66)
Session = 'Session02'
directory_path = 'C:/Users/jocu0013/Desktop/Oulanka/Scan_Raw/Oulanka2025_July/'
#directory_path ='C:/Users/jocu0013/Desktop/PhenoLinkY2/Root Scans/2024/Session009/'
path_out = 'C:/Users/jocu0013/Desktop/blended_images/AffineStitcher/2025_02/'+ 'Oulanka' + year + '_' + Session + '_' 
#path_out = 'C:/Users/jocu0013/Desktop/blended_images/AffineStitcher/2024_009/'+ 'Oulanka' + year + '_' + Session + '_' 
pattern = '.tiff'
imgs = list_files_with_pattern(directory_path,pattern)
# filter? 
#imgs = imgs[0:62] # only forest
imgs = imgs[63:274] # only fen

In [ ]:
def list_unique_names_within_range(names_list, start_index, end_index):
    unique_names = set()

    for name in names_list:
        subset_name = name[start_index:end_index]
        unique_names.add(subset_name)
        distinct_elements = list(set(unique_names))

    return list(distinct_elements)


def subset_strings_by_range(input_list, start_index, end_index):
    subset_strings = [element[start_index:end_index] for element in input_list if isinstance(element, str)]
    return subset_strings

def elements_matching_pattern(input_list, pattern):
    matching_elements = [element for element in input_list if pattern in element]
    return matching_elements



In [ ]:
imgs

In [ ]:
## extract names from file names

#unique_names = list_unique_names_within_range(imgs,75,79) #June 2023
unique_names = list_unique_names_within_range(imgs,73,77) # June 2022; July2022; June2023, June 2025
#unique_names = list_unique_names_within_range(imgs,76,80) # October 2022; # October 2023
#unique_names = list_unique_names_within_range(imgs,78,82) # September 2023
#unique_names = list_unique_names_within_range(imgs,89,93) # Spring 2023
#unique_names = list_unique_names_within_range(imgs,79,83) # Spring 2023 / 3
#unique_names = list_unique_names_within_range(imgs,80,84) # Spring 2024
#unique_names = list_unique_names_within_range(imgs,90,94) # Spring 2023
unique_names.sort()


#date = subset_strings_by_range(imgs,67,75)
#tube = subset_strings_by_range(imgs,57,61)

In [ ]:
unique_names

In [ ]:
from stitching import AffineStitcher
#import stitching

## play with the settings; hard to find good documentation on what some of them do; documentation for C++ could me informative too

#print(AffineStitcher.DEFAULT_SETTINGS)
#[print(f'{x}:\t{AffineStitcher.DEFAULT_SETTINGS[x]}') for x in AffineStitcher.DEFAULT_SETTINGS]
settings = {# chops everything not fitting in a rectangular
            "crop": False,
            # The matches confidences aren't that good
            "confidence_threshold": 0.0001,
            #'match_conf':0.5,
             'refinement_mask': 'x_x_x',            
             #'range_width': -0.1,
             'nfeatures': 100000000,
             #'low_megapix':5.0 ,
             #'medium_megapix': 12,
             'detector': "sift"}    

## affine for scans
stitcher = AffineStitcher(**settings) 
## other needed for perspectiive warping
#stitcher = stitching.Stitcher(**settings)
 

In [ ]:
AffineStitcher.DEFAULT_SETTINGS

In [ ]:
stitcher.settings

In [ ]:
for i in unique_names:
    #print(str(i))
    matching_images = elements_matching_pattern(imgs, str(i))
    path = path_out + str(i) + '.png'
    #path_top = 'C:/Users/jocu0013/Desktop/blended_images/'+ 'Oulanka' + str(year) + '_' + Session + '_' + str(i) +'_top' + '.png'
    #path_mid = 'C:/Users/jocu0013/Desktop/blended_images/'+ 'Oulanka' + str(year) + '_' + Session + '_' + str(i) +'_mid' + '.png'
    #path_bot = 'C:/Users/jocu0013/Desktop/blended_images/'+ 'Oulanka' + str(year) + '_' + Session + '_' + str(i) +'_bot' + '.png'
    print(path)
    
    ## two scanner windows
    if len(matching_images)==2:
        panorama = stitcher.stitch(matching_images)
        cv2.imwrite(path,panorama)
    ## single scanner window -> read & write without stitching
    if len(matching_images)==1:
        #doubling_image = matching_images + matching_images
        panorama = cv2.imread(matching_images[0])
        cv2.imwrite(path,panorama)
    else:
        # all six (potentially sequential stitching?! )
        try:
            panorama = stitcher.stitch(matching_images)
            cv2.imwrite(path,panorama)
        except:# StitchingError:
            print(f'{i} didnt work for some reason')
        


In [ ]:
# Two ways of failing
#1: StitchingError: No match exceeds the given confidence threshold.
#2: Excluding images from a stitching set without warning - check dimensions